<a href="https://colab.research.google.com/github/Pennyvia/trials/blob/main/SimpleChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data_root='/content/drive/My Drive/Chatbot'

Mounted at /content/drive


In [ ]:
import json
import string
import random
!pip install tensorflow
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import Dropout
nltk.download("punkt")
nltk.download("wordnet")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
data_file = open(data_root + '/intents.json').read()
data = json.loads(data_file)

In [ ]:
import nltk
nltk.download('all')
words = [] #BOW MODEL/ VOCABULARY for patterns
classes =[] #BOW model /VOCABULARY tags
data_x = [] #storing each pattern
data_y = [] #storing each tag for a corresponding pattern

#iterating over data
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        data_x.append(pattern)
        data_y.append(intent["tag"])

    if intent["tag"] not in classes:
      classes.append(intent["tag"])

lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(word.lower())for word in words if word not in string.punctuation]

words = sorted(set(words))
classes = sorted(set(classes))


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

In [ ]:
#converting text to numbers
training = []
out_empty = [0] * len(classes)

#creating BOW.
for idx, doc in enumerate(data_x):
  bow = []
  text = lemmatizer.lemmatize(doc.lower())
  for word in words:
     bow.append(1) if word in text else bow.append(0)
  output_row = list(out_empty)
  output_row[classes.index(data_y[idx])] = 1

  #add the encoded BOW and associated classes to training
  training.append([bow, output_row])

#shuffling the data and converting it to array
random.shuffle(training)
training = np.array(training, dtype=object)

#split the features and target labels
train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))



In [ ]:
#Neural Network Model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy',
             optimizer=adam,
             metrics=["accuracy"])
print(model.summary())
model.fit(x=train_x, y=train_y, epochs=150, verbose=1)




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               7936      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 11)                715       
                                                                 
Total params: 16,907
Trainable params: 16,907
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/150
2/2 [==============================

In [ ]:
#preprocessing the Input

def clean_text(text):
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

def bag_of_words(text, vocab):
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens:
    for idx, word in enumerate(vocab):
      if word == w:
        bow[idx] = 1
  return np.array(bow)

def pred_class(text, vocab, labels):
  bow = bag_of_words(text,vocab)
  result = model.predict(np.array([bow]))[0] #extracting probabilities
  thresh = 0.5
  y_pred = [[indx, res] for indx, res in enumerate(result) if res > thresh]
  y_pred.sort(key=lambda x: x[1], reverse=True) #sorting the values of probability in decreasing order
  return_list = []
  for r in y_pred:
    return_list.append(labels[r[0]]) #contains labels for highest probability
  return return_list

def get_response(intents_list, intents_json):
  if len(intents_list) == 0:
    result = "Sorry! I don't understand"
  else:
    tag = intents_list[0]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents:
      if i["tag"] == tag:
        result = random.choice(i["responses"])
        break
  return result


In [ ]:
print("Press 0 if you don't want to talk to our chatbot")

while True:
  message = input("")
  if message == "0":
    break
  intents = pred_class(message, words, classes)
  result = get_response(intents, data)
  print(result)

Press 0 if you don't want to talk to our chatbot
hi
1/1 [==============================] - 0s 277ms/step
Hi there, how can I help?
i have a problem
1/1 [==============================] - 0s 21ms/step
Explain your problem to me Dear
I have a headache
1/1 [==============================] - 0s 31ms/step
Remember your laws of health, check and assess which of them have you violated
I have a headache
1/1 [==============================] - 0s 17ms/step
you need to take a warm bath and take a nap for a while 
I have a headache
1/1 [==============================] - 0s 21ms/step
Remember your laws of health, check and assess which of them have you violated
My body is weak
1/1 [==============================] - 0s 21ms/step
you need to take a warm bath and take a nap for a while 
My body is weak
1/1 [==============================] - 0s 22ms/step
Remember your laws of health, check and assess which of them have you violated
What is your name
1/1 [==============================] - 0s 20ms/step
I